<a href="https://colab.research.google.com/github/smpark0520/ESAA/blob/main/3_14_%EC%84%B8%EC%85%98_%EB%AA%A8%EB%8D%B8%ED%9B%88%EB%A0%A8_%EC%97%B0%EC%8A%B5%EB%AC%B8%EC%A0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **모델 훈련 연습 문제**
___
- 출처 : 핸즈온 머신러닝 Ch04 연습문제 1, 5, 9, 10
- 개념 문제의 경우 텍스트 셀을 추가하여 정답을 적어주세요.

### **1. 수백만 개의 특성을 가진 훈련 세트에서는 어떤 선형 회귀 알고리즘을 사용할 수 있을까요?**
___


확률적 경사 하강법, 미니배치 경사 하강법, 배치 경사 하강법

### **2. 배치 경사 하강법을 사용하고 에포크마다 검증 오차를 그래프로 나타내봤습니다. 검증 오차가 일정하게 상승되고 있다면 어떤 일이 일어나고 있는 걸까요? 이 문제를 어떻게 해결할 수 있나요?**
___


학습률 줄이기

모델을 복잡하게 만들기

- 수렴이 아닌 발산 중인 것 -> 학습율이 커서 오차가 커지는 것
- 과소적합 (모델이 단순해서 오차가 클 수 있다)


### **3. 릿지 회귀를 사용했을 때 훈련 오차가 검증 오차가 거의 비슷하고 둘 다 높았습니다. 이 모델에는 높은 편향이 문제인가요, 아니면 높은 분산이 문제인가요? 규제 하이퍼파라미터 $\alpha$를 증가시켜야 할까요 아니면 줄여야 할까요?**
___

높은 편향이 문제이다

$alpha$ 를 줄여야한다

### **4. 다음과 같이 사용해야 하는 이유는?**
___
- 평범한 선형 회귀(즉, 아무런 규제가 없는 모델) 대신 릿지 회귀
- 릿지 회귀 대신 라쏘 회귀
- 라쏘 회귀 대신 엘라스틱넷

- 평범한 선형 회귀(즉, 아무런 규제가 없는 모델) 대신 릿지 회귀 : 다중 공산성 문제 해결을 위해 릿지 회귀를 사용한다.

- 릿지 회귀 대신 라쏘 회귀  : 중요하지 않은 것의 가중치를 0으로 만들어서 중요한 것만 추출하여 사용 가능하다.


- 라쏘 회귀 대신 엘라스틱넷 : 사용하는 특성이 적으면 라쏘 대신 엘라스틱넷 회귀를 이용할 수 있다.

### **추가) 조기 종료를 사용한 배치 경사 하강법으로 iris 데이터를 활용해 소프트맥스 회귀를 구현해보세요(사이킷런은 사용하지 마세요)**


---



In [2]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

# 1. 데이터 로드 및 전처리
iris = load_iris()
X = iris.data  # (150, 4) - 특성
y = iris.target  # (150,) - 타겟

# 원-핫 인코딩 (y를 one-hot 벡터로 변환)
encoder = OneHotEncoder(sparse_output=False)
y_one_hot = encoder.fit_transform(y.reshape(-1, 1))

# 훈련/검증 세트 분할
X_train, X_val, y_train, y_val = train_test_split(X, y_one_hot, test_size=0.2, random_state=42)

# 2. 소프트맥스 함수 정의
def softmax(z):
    exp_z = np.exp(z - np.max(z, axis=1, keepdims=True))  # Overflow 방지
    return exp_z / np.sum(exp_z, axis=1, keepdims=True)

# 3. 비용 함수 (Cross-Entropy Loss)
def cross_entropy_loss(y_true, y_pred):
    return -np.mean(np.sum(y_true * np.log(y_pred + 1e-8), axis=1))  # log(0) 방지

# 4. 배치 경사 하강법 + 조기 종료
def softmax_regression(X_train, y_train, X_val, y_val, learning_rate=0.1, epochs=1000, tol=1e-4, patience=10):
    m, n = X_train.shape  # 샘플 개수, 특성 개수
    k = y_train.shape[1]  # 클래스 개수
    W = np.random.randn(n, k) * 0.01  # 가중치 초기화
    b = np.zeros((1, k))  # 편향 초기화

    best_loss = float("inf")
    wait = 0  # 조기 종료 카운터

    for epoch in range(epochs):
        # 선형 조합
        logits = X_train @ W + b
        y_pred = softmax(logits)

        # 비용 계산
        loss = cross_entropy_loss(y_train, y_pred)

        # 기울기 계산
        dW = (X_train.T @ (y_pred - y_train)) / m
        db = np.mean(y_pred - y_train, axis=0, keepdims=True)

        # 가중치 업데이트
        W -= learning_rate * dW
        b -= learning_rate * db

        # 검증 세트에서 성능 평가
        val_logits = X_val @ W + b
        y_val_pred = softmax(val_logits)
        val_loss = cross_entropy_loss(y_val, y_val_pred)

        # 조기 종료 조건 확인
        if val_loss < best_loss - tol:
            best_loss = val_loss
            wait = 0
        else:
            wait += 1
            if wait >= patience:
                print(f"조기 종료 발생 (Epoch {epoch+1})")
                break

        # 100 에포크마다 진행상황 출력
        if epoch % 100 == 0:
            print(f"Epoch {epoch}: Train Loss={loss:.4f}, Val Loss={val_loss:.4f}")

    return W, b

# 5. 모델 훈련
W, b = softmax_regression(X_train, y_train, X_val, y_val)

# 6. 모델 예측 및 정확도 평가
def predict(X, W, b):
    return np.argmax(softmax(X @ W + b), axis=1)

y_pred_train = predict(X_train, W, b)
y_pred_val = predict(X_val, W, b)

train_acc = np.mean(y_pred_train == np.argmax(y_train, axis=1))
val_acc = np.mean(y_pred_val == np.argmax(y_val, axis=1))

print(f"훈련 정확도: {train_acc:.4f}, 검증 정확도: {val_acc:.4f}")

Epoch 0: Train Loss=1.1175, Val Loss=1.0380
Epoch 100: Train Loss=0.4436, Val Loss=0.4304
Epoch 200: Train Loss=0.2627, Val Loss=0.2619
Epoch 300: Train Loss=0.2190, Val Loss=0.2205
Epoch 400: Train Loss=0.1913, Val Loss=0.1942
Epoch 500: Train Loss=0.1720, Val Loss=0.1760
Epoch 600: Train Loss=0.1578, Val Loss=0.1628
Epoch 700: Train Loss=0.1469, Val Loss=0.1526
Epoch 800: Train Loss=0.1382, Val Loss=0.1445
Epoch 900: Train Loss=0.1312, Val Loss=0.1380
훈련 정확도: 0.9750, 검증 정확도: 1.0000
